In [1]:
import gym
import numpy as np
import tensorflow as tf

In [35]:
TOTAL_STEPS = 5000
TRAINING_STEPS = 3000
VALIDATION_STEPS = 1000
BATCH_SIZE = 64

## Cartpole-v0 Info
### Action Space
    The action is a `ndarray` with shape `(1,)` which can take values `{0, 1}` indicating the direction of the fixed force the cart is pushed with.
    | Num | Action                 |
    |-----|------------------------|
    | 0   | Push cart to the left  |
    | 1   | Push cart to the right |
    
### Observation Space
    The observation is a `ndarray` with shape `(4,)` with the values corresponding to the following positions and velocities:
    | Num | Observation           | Min                  | Max                |
    |-----|-----------------------|----------------------|--------------------|
    | 0   | Cart Position         | -4.8                 | 4.8                |
    | 1   | Cart Velocity         | -Inf                 | Inf                |
    | 2   | Pole Angle            | ~ -0.418 rad (-24°)  | ~ 0.418 rad (24°)  |
    | 3   | Pole Angular Velocity | -Inf                 | Inf                |
    
(https://github.com/openai/gym/blob/master/gym/envs/classic_control/cartpole.py)

In [71]:
env = gym.make('CartPole-v0')
X = np.empty((TOTAL_STEPS, 4)) # len(env.observation_space.sample()) == 4
U = np.empty((TOTAL_STEPS, 1))
y = np.empty((TOTAL_STEPS, 4))

In [117]:
tf.random.set_seed(1)
np.random.seed(1)

In [73]:
#### TEST ####
x_present = env.reset()
for timestep in range(1):
    u = env.action_space.sample() # sidenote: should this be one-hot encoded?
    x_prime, _, done, _ = env.step(u)
    
    # sample x[n], u[n], and x[n+1]
    X[timestep] = x_present
    U[timestep] = u
    y[timestep] = x_prime
    x_present = x_prime
print(X[0])
print(U[0])
print(y[0])

[-0.04416447  0.03527697 -0.00501254  0.04463949]
[1.]
[-0.04345893  0.23047043 -0.00411975 -0.24962072]


In [74]:
#### "Let-it-run method" ####
for iteration in range(TOTAL_STEPS // 1000):
    x_present = env.reset()
    for timestep in range(1000):
        u = env.action_space.sample() # sidenote: should this be one-hot encoded?
        x_prime, _, done, _ = env.step(u)

        # sample x[n], u[n], and x[n+1]
        X[1000 * iteration + timestep] = x_present
        U[1000 * iteration + timestep] = u
        y[1000 * iteration + timestep] = x_prime
        x_present = x_prime
env.close()

/Users/johnlime/anaconda3/lib/python3.8/site-packages/gym/envs/classic_control/cartpole.py:150: UserWarning: WARN: You are calling 'step()' even though this environment has already returned done = True. You should always call 'reset()' once you receive 'done = True' -- any further steps are undefined behavior.
  logger.warn(


In [118]:
x_present = env.reset()
for num_timestep in range(TOTAL_STEPS):
    u = env.action_space.sample() # sidenote: should this be one-hot encoded?
    x_prime, _, done, _ = env.step(u)

    # sample x[n], u[n], and x[n+1]
    X[num_timestep] = x_present
    U[num_timestep] = u
    y[num_timestep] = x_prime
    x_present = x_prime
    if done:
        x_present = env.reset()
env.close()

In [119]:
class CartpoleLinearControlSystem(tf.keras.Model):
    def __init__(self):
        super(CartpoleLinearControlSystem, self).__init__()
        self.A = tf.keras.layers.Dense(4, input_shape = (4, ), activation = 'linear', name = 'A')
        self.B = tf.keras.layers.Dense(4, input_shape = (1, ), activation = 'linear', name = 'B')
        self.add = tf.keras.layers.Add()
        
    def call(self, inputs):
        x, u = inputs
        return self.A(x) + self.B(u)

Uses reference code in *Python Machine Learning: 3rd Edition* (Raschka and Mirjalili, 2019) pp. 438, 512-515

In [120]:
# create tensorflow dataset object instance; concat X and y
dataset_xu = tf.data.Dataset.from_tensors((X, U))
dataset_y = tf.data.Dataset.from_tensors(y)
dataset = tf.data.Dataset.zip((dataset_xu, dataset_y)).shuffle(1000)
print(dataset)
# split trajectories into training, validation, and testing
dataset_training = dataset.take(TRAINING_STEPS)
dataset_validation = dataset.skip(TRAINING_STEPS).take(VALIDATION_STEPS)
dataset_testing = dataset.skip(TRAINING_STEPS + VALIDATION_STEPS)
# split training data into batches
dataset_training = dataset_training.repeat(5).batch(BATCH_SIZE)

<ShuffleDataset shapes: (((5000, 4), (5000, 1)), (5000, 4)), types: ((tf.float64, tf.float64), tf.float64)>


In [121]:
# compile model
model = CartpoleLinearControlSystem()
model.compile(tf.keras.optimizers.Adam(),
              loss = tf.keras.losses.MeanSquaredError(),
              metrics = [tf.keras.metrics.MeanSquaredError()])
history = model.fit(dataset_training, validation_data = dataset_validation, epochs = 1000)
tf.keras.models.save_model(
    model, './cartpole_system_model', overwrite = True, include_optimizer = True, save_format = None,
    signatures = None, options = None, save_traces = True
)

Epoch 1/1000
1/1 [==============================] - 0s 429ms/step - loss: 0.8100 - mean_squared_error: 0.8100
Epoch 2/1000
1/1 [==============================] - 0s 17ms/step - loss: 0.8072 - mean_squared_error: 0.8072
Epoch 3/1000
1/1 [==============================] - 0s 16ms/step - loss: 0.8044 - mean_squared_error: 0.8044
Epoch 4/1000
1/1 [==============================] - 0s 18ms/step - loss: 0.8016 - mean_squared_error: 0.8016
Epoch 5/1000
1/1 [==============================] - 0s 16ms/step - loss: 0.7988 - mean_squared_error: 0.7988
Epoch 6/1000
1/1 [==============================] - 0s 18ms/step - loss: 0.7961 - mean_squared_error: 0.7961
Epoch 7/1000
1/1 [==============================] - 0s 17ms/step - loss: 0.7934 - mean_squared_error: 0.7934
Epoch 8/1000
1/1 [==============================] - 0s 18ms/step - loss: 0.7907 - mean_squared_error: 0.7907
Epoch 9/1000
1/1 [==============================] - 0s 16ms/step - loss: 0.7880 - mean_squared_error: 0.7880
Epoch 10/1000
1/1 

1/1 [==============================] - 0s 19ms/step - loss: 0.6385 - mean_squared_error: 0.6385
Epoch 76/1000
1/1 [==============================] - 0s 17ms/step - loss: 0.6367 - mean_squared_error: 0.6367
Epoch 77/1000
1/1 [==============================] - 0s 16ms/step - loss: 0.6348 - mean_squared_error: 0.6348
Epoch 78/1000
1/1 [==============================] - 0s 19ms/step - loss: 0.6329 - mean_squared_error: 0.6329
Epoch 79/1000
1/1 [==============================] - 0s 17ms/step - loss: 0.6310 - mean_squared_error: 0.6310
Epoch 80/1000
1/1 [==============================] - 0s 17ms/step - loss: 0.6292 - mean_squared_error: 0.6292
Epoch 81/1000
1/1 [==============================] - 0s 17ms/step - loss: 0.6273 - mean_squared_error: 0.6273
Epoch 82/1000
1/1 [==============================] - 0s 18ms/step - loss: 0.6255 - mean_squared_error: 0.6255
Epoch 83/1000
1/1 [==============================] - 0s 17ms/step - loss: 0.6237 - mean_squared_error: 0.6237
Epoch 84/1000
1/1 [=====

1/1 [==============================] - 0s 16ms/step - loss: 0.5184 - mean_squared_error: 0.5184
Epoch 150/1000
1/1 [==============================] - 0s 18ms/step - loss: 0.5170 - mean_squared_error: 0.5170
Epoch 151/1000
1/1 [==============================] - 0s 19ms/step - loss: 0.5157 - mean_squared_error: 0.5157
Epoch 152/1000
1/1 [==============================] - 0s 17ms/step - loss: 0.5143 - mean_squared_error: 0.5143
Epoch 153/1000
1/1 [==============================] - 0s 16ms/step - loss: 0.5129 - mean_squared_error: 0.5129
Epoch 154/1000
1/1 [==============================] - 0s 18ms/step - loss: 0.5115 - mean_squared_error: 0.5115
Epoch 155/1000
1/1 [==============================] - 0s 16ms/step - loss: 0.5101 - mean_squared_error: 0.5101
Epoch 156/1000
1/1 [==============================] - 0s 17ms/step - loss: 0.5088 - mean_squared_error: 0.5088
Epoch 157/1000
1/1 [==============================] - 0s 17ms/step - loss: 0.5074 - mean_squared_error: 0.5074
Epoch 158/1000
1

1/1 [==============================] - 0s 16ms/step - loss: 0.4279 - mean_squared_error: 0.4279
Epoch 223/1000
1/1 [==============================] - 0s 15ms/step - loss: 0.4268 - mean_squared_error: 0.4268
Epoch 224/1000
1/1 [==============================] - 0s 17ms/step - loss: 0.4257 - mean_squared_error: 0.4257
Epoch 225/1000
1/1 [==============================] - 0s 17ms/step - loss: 0.4246 - mean_squared_error: 0.4246
Epoch 226/1000
1/1 [==============================] - 0s 18ms/step - loss: 0.4235 - mean_squared_error: 0.4235
Epoch 227/1000
1/1 [==============================] - 0s 16ms/step - loss: 0.4224 - mean_squared_error: 0.4224
Epoch 228/1000
1/1 [==============================] - 0s 17ms/step - loss: 0.4213 - mean_squared_error: 0.4213
Epoch 229/1000
1/1 [==============================] - 0s 16ms/step - loss: 0.4202 - mean_squared_error: 0.4202
Epoch 230/1000
1/1 [==============================] - 0s 26ms/step - loss: 0.4191 - mean_squared_error: 0.4191
Epoch 231/1000
1

1/1 [==============================] - 0s 16ms/step - loss: 0.3547 - mean_squared_error: 0.3547
Epoch 296/1000
1/1 [==============================] - 0s 16ms/step - loss: 0.3538 - mean_squared_error: 0.3538
Epoch 297/1000
1/1 [==============================] - 0s 17ms/step - loss: 0.3528 - mean_squared_error: 0.3528
Epoch 298/1000
1/1 [==============================] - 0s 16ms/step - loss: 0.3519 - mean_squared_error: 0.3519
Epoch 299/1000
1/1 [==============================] - 0s 16ms/step - loss: 0.3510 - mean_squared_error: 0.3510
Epoch 300/1000
1/1 [==============================] - 0s 20ms/step - loss: 0.3501 - mean_squared_error: 0.3501
Epoch 301/1000
1/1 [==============================] - 0s 18ms/step - loss: 0.3492 - mean_squared_error: 0.3492
Epoch 302/1000
1/1 [==============================] - 0s 17ms/step - loss: 0.3483 - mean_squared_error: 0.3483
Epoch 303/1000
1/1 [==============================] - 0s 20ms/step - loss: 0.3475 - mean_squared_error: 0.3475
Epoch 304/1000
1

...

1/1 [==============================] - 0s 17ms/step - loss: 0.0722 - mean_squared_error: 0.0722
Epoch 880/1000
1/1 [==============================] - 0s 16ms/step - loss: 0.0719 - mean_squared_error: 0.0719
Epoch 881/1000
1/1 [==============================] - 0s 19ms/step - loss: 0.0717 - mean_squared_error: 0.0717
Epoch 882/1000
1/1 [==============================] - 0s 16ms/step - loss: 0.0715 - mean_squared_error: 0.0715
Epoch 883/1000
1/1 [==============================] - 0s 17ms/step - loss: 0.0713 - mean_squared_error: 0.0713
Epoch 884/1000
1/1 [==============================] - 0s 16ms/step - loss: 0.0711 - mean_squared_error: 0.0711
Epoch 885/1000
1/1 [==============================] - 0s 16ms/step - loss: 0.0709 - mean_squared_error: 0.0709
Epoch 886/1000
1/1 [==============================] - 0s 17ms/step - loss: 0.0707 - mean_squared_error: 0.0707
Epoch 887/1000
1/1 [==============================] - 0s 16ms/step - loss: 0.0705 - mean_squared_error: 0.0705
Epoch 888/1000
1

1/1 [==============================] - 0s 16ms/step - loss: 0.0583 - mean_squared_error: 0.0583
Epoch 953/1000
1/1 [==============================] - 0s 16ms/step - loss: 0.0582 - mean_squared_error: 0.0582
Epoch 954/1000
1/1 [==============================] - 0s 17ms/step - loss: 0.0580 - mean_squared_error: 0.0580
Epoch 955/1000
1/1 [==============================] - 0s 17ms/step - loss: 0.0578 - mean_squared_error: 0.0578
Epoch 956/1000
1/1 [==============================] - 0s 18ms/step - loss: 0.0577 - mean_squared_error: 0.0577
Epoch 957/1000
1/1 [==============================] - 0s 15ms/step - loss: 0.0575 - mean_squared_error: 0.0575
Epoch 958/1000
1/1 [==============================] - 0s 17ms/step - loss: 0.0573 - mean_squared_error: 0.0573
Epoch 959/1000
1/1 [==============================] - 0s 17ms/step - loss: 0.0572 - mean_squared_error: 0.0572
Epoch 960/1000
1/1 [==============================] - 0s 16ms/step - loss: 0.0570 - mean_squared_error: 0.0570
Epoch 961/1000
1

2022-03-06 04:21:13.492885: W tensorflow/python/util/util.cc:368] Sets are not currently considered sequences, but this may change in the future, so consider avoiding using them.


INFO:tensorflow:Assets written to: ./cartpole_system_model/assets


/Users/johnlime/anaconda3/lib/python3.8/site-packages/keras/saving/saved_model/layer_serialization.py:112: CustomMaskWarning: Custom mask layers require a config and must override get_config. When loading, the custom mask layer must be passed to the custom_objects argument.
  return generic_utils.serialize_keras_object(obj)


In [122]:
print(history)